# Stock Trading Strategy Based on Neural Net Time Series Analysis

For background, please see our previous notebook: leading_pattern/notebook.ipynb. Also, the work below is based on the ideas from [this page](https://www.tensorflow.org/tutorials/structured_data/time_series) on tensorflow.org

The basic idea is to predict the price of a target stock based on historical values (price, volume, etc..) of a set of predictor stocks; an event happens, it ripples through predictors then to the target. In our previous strategy we found this is difficult... maybe even impossible. Maybe we will have better luck with neural networks (NN).

One possible benefit of NN is they make it easy to combine predictors. Previously we looked for individual stocks as predictors with the thought that we would find a way to combine them. With that method it is even possible we could have two or more predictors that by themselves could not exceed a threshold, but together they would. With NN we would find such a predictor.

In the previous analysis, we looked for spikes in the target that lasted 2 days and exceeded a 3% gain. That approach gave us about 10-20 events/year for an average stock. Using a NN classifier on data like that would fail because the algorithm would right 90% of the time just by predicting "no event" everyday. To get around that issue (kick the can further down the road?) we will try to predict the price tomorrow for every day. If the price tomorrow is some threshold above today, we would buy. After that, we would sell if the price was predicted to go down.

One potential drawback of the approach is we could tie our money up on small, single day price swings. However with our previous strategy we also tied our money up on false positives.

We could design a NN to do multi-day predictions, but at this point it is not clear we will even be able to do single-day predictions. So lets try to walk before we run.

## The Baseline Strategy
A simple and robust strategy is simply to say the price tomorrow will be the same as today. In fact, in a bull market, that strategy might make a lot of money.

Setting aside the idea of making money, this strategy also provide a baseline for how well we can predict prices. If we can't do significantly better than that, then our predictor is useless.

## Linear Model
With this model, we are saying that the price of the target tomorrow is a weighted sum of the predictors today.

In [4]:
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from nn import nn
from nn.nn import MAX_EPOCHS


plt.rcParams['figure.figsize'] = [10, 5]

config = {
    'start_date': '20170103',
    'end_date': '20181231',
    'price_field': 'Open',
    'predictor_field': 'Open'
}
splits = [0.5, 0.75]

# Load some stocks
target = 'NPTN'  # this is automatically included in the predictors

# These are the ticker symbols for a random selection of stocks
predictors = ['MTL', 'AGR', 'PHD', 'ESS', 'CBOE', 'AGRO', 'FPF', 'CHMA', 'SEAS']
n_predictors = len(predictors) + 1
df = nn.load_data(target, predictors, config)
norm_and_split = nn.NormalizeAndSplit(df, splits, 'target')
train_df, val_df, test_df = norm_and_split.moving_avg(10)
mean_absolute_error = []
column_indices = {name: i for i, name in enumerate(df.columns)}

# Make a window that has one time as an input, one time as an output with a shift of one between them
input_width = 1
label_width = 1
shift = 1
single_step_window = \
    nn.WindowGenerator(input_width, label_width, shift, train_df, val_df, test_df, label_columns=['target'])

# Run baseline model to get performance
baseline = nn.Baseline(label_index=column_indices['target'])
baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanAbsoluteError()])

# We do not need to "train" this model because it simply returns the current value as the prediction
perf = [
    'Baseline',
    baseline.evaluate(single_step_window.train, verbose=0)[1],
    baseline.evaluate(single_step_window.val, verbose=0)[1],
    baseline.evaluate(single_step_window.test, verbose=0)[1]
]
mean_absolute_error.append(perf)

# This is the linear model
linear = tf.keras.Sequential([tf.keras.layers.Dense(units=1)])
nn.compile_and_fit(linear, single_step_window, max_epochs=100, patience=10)

perf = [
    'Linear',
    linear.evaluate(single_step_window.train, verbose=0)[1],
    linear.evaluate(single_step_window.val, verbose=0)[1],
    linear.evaluate(single_step_window.test, verbose=0)[1]
]
mean_absolute_error.append(perf)

single_step_window.plot_fit(linear, norm_and_split.de_normalize, 'Linear')

in call
Epoch 1/100
8/8 [==============================] - 0s 16ms/step - loss: 0.0042 - mean_absolute_error: 0.0488 - val_loss: 0.0040 - val_mean_absolute_error: 0.0515
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0041 - mean_absolute_error: 0.0478 - val_loss: 0.0039 - val_mean_absolute_error: 0.0506
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0040 - mean_absolute_error: 0.0473 - val_loss: 0.0038 - val_mean_absolute_error: 0.0501
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0039 - mean_absolute_error: 0.0468 - val_loss: 0.0037 - val_mean_absolute_error: 0.0498
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0039 - mean_absolute_error: 0.0464 - val_loss: 0.0037 - val_mean_absolute_error: 0.0495
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0038 - mean_absolute_error: 0.0460 - val_loss: 0.0036 - val_mean_absolute_error: 0.0492
Epoch 7/100
8/8 [========

8/8 [==============================] - 0s 7ms/step - loss: 0.0022 - mean_absolute_error: 0.0332 - val_loss: 0.0023 - val_mean_absolute_error: 0.0386
Epoch 52/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0022 - mean_absolute_error: 0.0330 - val_loss: 0.0023 - val_mean_absolute_error: 0.0385
Epoch 53/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0022 - mean_absolute_error: 0.0329 - val_loss: 0.0023 - val_mean_absolute_error: 0.0384
Epoch 54/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0021 - mean_absolute_error: 0.0327 - val_loss: 0.0023 - val_mean_absolute_error: 0.0382
Epoch 55/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0021 - mean_absolute_error: 0.0325 - val_loss: 0.0023 - val_mean_absolute_error: 0.0381
Epoch 56/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0021 - mean_absolute_error: 0.0324 - val_loss: 0.0023 - val_mean_absolute_error: 0.0380
Epoch 57/100
8/8 [=======================

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
print(pd.DataFrame(mean_absolute_error, columns=['Model', 'train', 'val', 'test']))

      Model     train       val      test
0  Baseline  0.028716  0.031093  0.028820
1    Linear  0.043044  0.050577  0.071071


Whether or not this is a good fit depends on context. If this were temperature and you were deciding to wear a coat, it might be a pretty good fit.

However, if you look closely, you will see that the predictions are always a day late. This is because one of the predictors is the target. The linear model could give the same results as the baseline model if the optimizer simply set the weight of the 